In [2]:
import pandas as pd
from pydataset import data
import seaborn as sns
from acquire import get_iris_data,get_telco_data,get_titantic_data

## 4. In a jupyter notebook, classification_exercises.ipynb, use a python module (pydata or seaborn datasets) containing datasets as a source from the iris data. Create a pandas dataframe, df_iris, from this data.

In [2]:
df_iris = data('iris')

print the first 3 rows

In [3]:
df_iris.head(3)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa


print the number of rows and columns (shape)

In [4]:
df_iris.shape

(150, 5)

print the column names

In [5]:
df_iris.columns

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width',
       'Species'],
      dtype='object')

print the data type of each column

In [6]:
df_iris.dtypes

Sepal.Length    float64
Sepal.Width     float64
Petal.Length    float64
Petal.Width     float64
Species          object
dtype: object

print the summary statistics for each of the numeric variables. Would you recommend rescaling the data based on these statistics?

In [7]:
df_iris.describe()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [8]:
# Scaling does not seem to be required as all data is in cm

## 5. Read the Table1_CustDetails table from your spreadsheet exercises google sheet into a dataframe named df_google_sheets.

In [9]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1kihBDi60lQX47cLTxNbLjnOnil0kLu922K7i5LM9hxE/edit#gid=1023018493'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df_google_sheets= pd.read_csv(csv_export_url)

assign the first 100 rows to a new dataframe, df_excel_sample

In [10]:
df_google_sheets_sample = df_google_sheets.head(100)

print the number of rows of your original dataframe

In [11]:
df_google_sheets.shape[0]

7049

print the first 5 column names

In [12]:
df_google_sheets.columns[:5]

Index(['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents'], dtype='object')

print the column names that have a data type of object

In [13]:
df_google_sheets.select_dtypes(include = 'object').columns

Index(['customer_id', 'gender', 'partner', 'dependents', 'payment_type',
       'churn'],
      dtype='object')

compute the range for each of the numeric variables.

In [14]:
df_google_sheets.select_dtypes(include = 'number').max()-df_google_sheets.select_dtypes(include = 'number').min()

is_senior_citizen       1.0
phone_service           2.0
internet_service        2.0
contract_type           2.0
monthly_charges       100.5
total_charges        8666.0
dtype: float64

## 6. Download your spreadsheet exercises google sheet as an excel file (File → Download → Microsoft Excel). Read the Table1_CustDetails worksheet into a dataframe named df_excel.


In [15]:
df_excel = pd.read_excel('innis_spreadsheet_exercises_marder.xlsx')

assign the first 100 rows to a new dataframe, df_excel_sample

In [16]:
df_excel_sample = df_excel.head(100)

print the number of rows of your original dataframe

In [17]:
df_excel.shape[0]

7049

print the first 5 column names

In [18]:
df_excel.columns[:5]

Index(['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents'], dtype='object')

print the column names that have a data type of object

In [19]:
df_excel.select_dtypes('object').columns

Index(['customer_id', 'gender', 'partner', 'dependents', 'payment_type',
       'churn'],
      dtype='object')

compute the range for each of the numeric variables.

In [20]:
df_excel.select_dtypes(include = 'number').max()-df_excel.select_dtypes(include = 'number').min()

is_senior_citizen       1.0
phone_service           2.0
internet_service        2.0
contract_type           2.0
monthly_charges       100.5
total_charges        8666.0
dtype: float64

## 7. Read the data from this google sheet into a dataframe, df_google

In [21]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1Uhtml8KY19LILuZsrDtlsHHDC9wuDGUSe8LTEwvdI5g/edit#gid=341089357'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df_google = pd.read_csv(csv_export_url)

print the first 3 rows

In [22]:
df_google.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


print the number of rows and columns

In [23]:
df_google.shape

(891, 12)

print the column names

In [24]:
df_google.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

print the data type of each column

In [25]:
df_google.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

print the summary statistics for each of the numeric variables

In [26]:
df_google.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


print the unique values for each of your categorical variables

In [27]:
for col in df_google[df_google.columns[df_google.nunique()<10]]:
    print(col,': ',df_google[col].unique())

Survived :  [0 1]
Pclass :  [3 1 2]
Sex :  ['male' 'female']
SibSp :  [1 0 3 4 2 5 8]
Parch :  [0 1 2 5 3 4 6]
Embarked :  ['S' 'C' 'Q' nan]


# Data Preparation Exercises

In [256]:
# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

### Prepare Iris Data

In [3]:
iris = get_iris_data()

Using cached data


In [4]:
iris.head()

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,1,5.1,3.5,1.4,0.2,setosa
1,1,2,4.9,3.0,1.4,0.2,setosa
2,1,3,4.7,3.2,1.3,0.2,setosa
3,1,4,4.6,3.1,1.5,0.2,setosa
4,1,5,5.0,3.6,1.4,0.2,setosa


In [5]:
iris = iris.drop(columns=['species_id','measurement_id'])

In [8]:
dummy_df = pd.get_dummies(iris[['species_name']], dummy_na=False, drop_first=[True])

In [11]:
iris = pd.concat([iris, dummy_df], axis = 1)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species_name,species_name_versicolor,species_name_virginica,species_name_versicolor,species_name_virginica
0,5.1,3.5,1.4,0.2,setosa,0,0,0,0
1,4.9,3.0,1.4,0.2,setosa,0,0,0,0
2,4.7,3.2,1.3,0.2,setosa,0,0,0,0
3,4.6,3.1,1.5,0.2,setosa,0,0,0,0
4,5.0,3.6,1.4,0.2,setosa,0,0,0,0


In [67]:
def prep_iris(iris):
    iris = iris.drop(columns=['species_id','measurement_id'])
    # dummy_df = pd.get_dummies(iris[['species_name']], dummy_na=False, drop_first=[True])
    # iris = pd.concat([iris, dummy_df], axis = 1)
    # iris = iris.drop(columns=['species_name'])

    return iris

In [68]:
iris = prep_iris(get_iris_data())

Using cached data


## Use Titantic dataset

In [239]:
df = get_titantic_data()

Using cached data


In [240]:
df = df.drop(columns=['passenger_id','embarked','deck','class','age'])

In [241]:
dummy_df = pd.get_dummies(df[['sex','embark_town']], dummy_na=False, drop_first=True)
df = pd.concat([df,dummy_df], axis = 1)

In [243]:
train, test = train_test_split(df,test_size=0.2,random_state=123, stratify=df.survived)

In [244]:
train.shape

(712, 11)

In [245]:
test.shape

(179, 11)

In [246]:
train, validate = train_test_split(train, test_size=0.3, random_state=123, stratify=train.survived)

In [248]:
validate.shape

(214, 11)

In [257]:
imputer = SimpleImputer(missing_values = np.nan, strategy='most_frequent')

In [266]:
imputer = imputer.fit(train[['embark_town']])
imputer.statistics_

array(['Southampton'], dtype=object)

In [273]:
train[['embark_town']] = imputer.transform(train[['embark_town']])
test[['embark_town']] = imputer.transform(test[['embark_town']])
validate[['embark_town']] = imputer.transform(validate[['embark_town']])

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [274]:
validate.embark_town.value_counts(dropna=False)

Southampton    160
Cherbourg       38
Queenstown      16
Name: embark_town, dtype: int64

In [320]:
def prep_titantic(df):
    df = df.drop(columns=['passenger_id','embarked','deck','class','age'])
    df.embark_town = df.embark_town.fillna('Southampton')
    dummy_df = pd.get_dummies(df[['sex','embark_town']], dummy_na=False, drop_first=True)
    df = pd.concat([df,dummy_df], axis = 1)
    
    return df.drop(columns=['sex','embark_town'])

In [321]:
prep_titantic(get_titantic_data())

Using cached data


,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,3,1,0,7.2500,0,1,0,1
1,1,1,1,0,71.2833,0,0,0,0
2,1,3,0,0,7.9250,1,0,0,1
3,1,1,1,0,53.1000,0,0,0,1
4,0,3,0,0,8.0500,1,1,0,1
...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.0000,1,1,0,1
887,1,1,0,0,30.0000,1,0,0,1
888,0,3,1,2,23.4500,0,0,0,1
889,1,1,0,0,30.0000,1,1,0,0


In [285]:
tdf.embark_town.fillna('Southampton').isna().sum()

0

## Use Telco Dataset

In [314]:
df = get_telco_data()

Using cached data


In [312]:
df = df.replace(" ",np.nan)

In [212]:
df.shape

(7043, 24)

In [191]:
df.columns

Index(['payment_type_id', 'internet_service_type_id', 'contract_type_id',
       'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type'],
      dtype='object')

In [192]:
df = df.drop(columns=['payment_type_id','internet_service_type_id','contract_type_id'])

In [195]:
catcol = df.columns[(df.nunique()<5)&(df.dtypes == 'object')]

In [200]:
dummy_df = pd.get_dummies(df[catcol], dummy_na=False, drop_first=True)

In [202]:
df = pd.concat([df,dummy_df],axis=1)

In [203]:
df.shape

(7043, 48)

In [204]:
df = df.drop(columns=catcol)

In [205]:
df.shape

(7043, 32)

In [318]:
def prep_telco(df):
    # replace whitespace only cells with nan
    df = df.replace(" ",np.nan)
    # Drop the rows with NAs 
    df = df.dropna()
    # Drop unnecessary foreign key ids
    df = df.drop(columns=['payment_type_id','internet_service_type_id','contract_type_id'])
    # Determine the categorical variables - here defined as object data type (non-numeric) and with fewer than 5 values
    catcol = df.columns[(df.nunique()<5)&(df.dtypes == 'object')]
    # Encode categoricals
    dummy_df = pd.get_dummies(df[catcol], dummy_na=False, drop_first=True)
    # Concatenate dummy df to original df
    df = pd.concat([df,dummy_df],axis=1)
    # Remove the original categorical columns after encoding
    df = df.drop(columns=catcol)
    
    return df

In [319]:
prep_telco(get_telco_data())

Using cached data


,customer_id,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_No phone service,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0002-ORFBO,0,9,65.60,593.3,0,1,1,1,0,...,0,1,0,1,0,0,0,0,0,1
1,0003-MKNFE,0,9,59.90,542.4,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2,0004-TLHLJ,0,4,73.90,280.85,1,0,0,1,0,...,0,1,1,0,0,1,0,0,1,0
3,0011-IGKFF,1,13,98.00,1237.85,1,1,0,1,0,...,1,1,1,0,0,1,0,0,1,0
4,0013-EXCHZ,1,3,83.90,267.4,0,1,0,1,0,...,0,1,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,0,13,55.15,742.9,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
7039,9992-RRAMN,0,22,85.10,1873.7,1,1,0,1,0,...,1,1,1,0,0,1,0,0,1,0
7040,9992-UJOEL,0,2,50.30,92.75,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7041,9993-LHIEB,0,67,67.85,4627.65,1,1,1,1,0,...,1,0,0,0,1,0,0,0,0,1
